In [10]:
import numpy as np
import matplotlib.pyplot as plt
import sys 

"""
Implement the different exploration strategies.

  * mab is a MAB (MultiArmedBandit) object as defined below
  * epsilon is a scalar, which influences the amount of random actions
  * schedule is a callable decaying epsilon

You can get the approximated Q-values via mab.bandit_q_values and the different
counters for the bandits via mab.bandit_counters. mab.no_actions gives you the number
of arms.
"""


In [22]:
#     actions: actions-1 to acitons-6
#     with prop: (1-epsilon), select the greedy action: action-i
#     with prop: epsilon, select a random action: action-1 - aciton-6
def epsilon_greedy(mab, epsilon):
    mab_counters = np.array(mab.bandit_counters()) #okay, all history of the action-1 - aciton-6
    mab_q_values = mab.bandit_q_values() #after updating the q-values, I select the one action with highest
    total_actions = mab.no_actions() #total number of arms
    prob = mab_q_values/np.sum(mab_q_values)
    for i in range(0, total_actions):
        prob[i] = (1-epsilon)*prob[i] #greedy selection
    random_arm = np.random.randint(0, total_actions, size=1)
#     raise NotImplementedError()
    

In [23]:
def decaying_epsilon_greedy(mab, epsilon_init, schedule):
    raise NotImplementedError()

In [24]:
def random(mab):
    raise NotImplementedError()

In [25]:
def ucb(mab, c):
    raise NotImplementedError()

In [26]:
def softmax(mab, tau):
    raise NotImplementedError()

In [27]:
class Bandit:
    def __init__(self, bias, q_value=0, counter=0):
        self.bias = bias
        self.q_value = q_value
        self.counter = counter

    def pull(self):
        self.counter += 1
        reward = np.clip(self.bias + np.random.uniform(), 0, 1)
        self.q_value = self.q_value + 1/self.counter * (reward - self.q_value)
        return reward

In [28]:
reward = np.clip(0.1 + np.random.uniform(), 0, 1)
import matplotlib.pyplot as plt
reward

0.6588399937670025

In [29]:
epsilon = 0.5
epsilon_init = 0.6
tau = 0.01
c = 1.0
strategies = {
    epsilon_greedy: {'epsilon': epsilon},
    decaying_epsilon_greedy: {'epsilon_init': epsilon_init},
    random: {},
    ucb: {'c': c},
    softmax: {'tau': tau}
  }
for strategy, parameters in strategies.items():
    print(" ",strategy, " ", parameters)

  <function epsilon_greedy at 0x112f59400>   {'epsilon': 0.5}
  <function decaying_epsilon_greedy at 0x112f59c80>   {'epsilon_init': 0.6}
  <function random at 0x112f59e18>   {}
  <function ucb at 0x112f59ea0>   {'c': 1.0}
  <function softmax at 0x112f59f28>   {'tau': 0.01}


In [30]:
class MAB:
    def __init__(self, best_action, *bandits):
        self.bandits = bandits
        self._no_actions = len(bandits)
        self.step_counter = 0
        self.best_action = best_action

    def pull(self, action):
        self.step_counter += 1
        return self.bandits[action].pull(), self.bandits[action].q_value

    def run(self, no_rounds, exploration_strategy, **strategy_parameters):
        regrets = []
        rewards = []
        for i in range(no_rounds):
            if (i + 1) % 100 == 0:
                print("\rRound {}/{}".format(i + 1, no_rounds), end="")
                sys.stdout.flush()
            action = exploration_strategy(self, **strategy_parameters)
            reward, q = self.pull(action)
            best_action_value = self.best_action(self)[1]
            regret = best_action_value - q
            regrets.append(regret)
            rewards.append(reward)
        return regrets, rewards

    @property
    def bandit_counters(self):
        return np.array([bandit.counter for bandit in self.bandits])

    @property
    def bandit_q_values(self):
        return np.array([bandit.q_value for bandit in self.bandits])

    @property
    def no_actions(self):
        return self._no_actions

In [31]:
def plot(regrets):
    for strategy, regret in regrets.items():
        total_regret = np.cumsum(regret)
        plt.ylabel('Total Regret')
        plt.xlabel('Rounds')
        plt.plot(np.arange(len(total_regret)), total_regret, label=strategy)
    plt.legend()
    plt.savefig('regret.pdf', bbox_inches='tight')

In [32]:
if __name__ == '__main__':
    no_rounds = 1000000
    def schedule(mab, epsilon_init):
        raise NotImplementedError()

    epsilon = 0.5
    epsilon_init = 0.6
    tau = 0.01
    c = 1.0

    strategies = {
        epsilon_greedy: {'epsilon': epsilon},
        decaying_epsilon_greedy: {'epsilon_init': epsilon_init, 'schedule': schedule},
        random: {},
        ucb: {'c': c},
        softmax: {'tau': tau}
    }

    average_total_returns = {}
    total_regrets = {}
    num_actions = 10
    biases = [1.0 / k for k in range(5, 5+num_actions)]
    best_action_index = 0
    best_action_value = 0.7
    def best_action(mab):
        return best_action_index, best_action_value
    for strategy, parameters in strategies.items():
        print(strategy.__name__)
        bandits = [Bandit(bias, 1-bias) for bias in biases]
        mab = MAB(best_action, *bandits)
        total_regret, average_total_return = mab.run(no_rounds, strategy, **parameters)
        print("\n")
        average_total_returns[strategy.__name__] = average_total_return
        total_regrets[strategy.__name__] = total_regret
    plot(total_regrets)

epsilon_greedy


TypeError: 'numpy.ndarray' object is not callable